# Jigsaw - Agile Community Rules Classification
### https://www.kaggle.com/competitions/jigsaw-agile-community-rules

## LLM-Based Content Moderation with LoRA Fine-Tuning
This notebook builds upon our earlier zero-shot inference experiments using a Llama-1B model for Reddit content moderation. Previously, we explored:

Token-based inference using Unsloth + vLLM: https://www.kaggle.com/code/vinothkumarsekar89/jigsaw-acrc-llama-1b-infer-automodel-unsloth-vllm

Logit-based batch inference using vLLM: https://www.kaggle.com/code/vinothkumarsekar89/jigsaw-acrc-llama-1b-batch-infer-logits-vllm

These approaches relied solely on zero-shot prompting without any model fine-tuning.

In this notebook, we fine-tune the Llama-1B model using LoRA (Low-Rank Adaptation) on the provided training dataset to improve classification performance. We then compare the fine-tuned model’s performance against the previous zero-shot baselines using F1-score metrics on a validation set Further... 

Inference using fine-tuned model will be available here: WIP

## Install packages on Kaggle: Add-ons > Install Dependencies 

```bash
pip install pip3-autoremove
pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
pip install unsloth vllm
pip install scikit-learn
```

In [1]:
import kagglehub
import pandas as pd
import os

# Check if running on Kaggle
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
   # Running on Kaggle
   base_path = "/kaggle/input/jigsaw-agile-community-rules/"
   df_train = pd.read_csv(f"{base_path}train.csv")
   df_test = pd.read_csv(f"{base_path}test.csv")
else:
   # Running locally
   base_path = "./data/"
   df_train = pd.read_csv(f"{base_path}train.csv")
   df_test = pd.read_csv(f"{base_path}test.csv")

print(f"Using path: {base_path}")
df_train.head(2)

Using path: ./data/


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0


## Set imports, variable names, parameters

In [2]:
from unsloth import FastLanguageModel
import torch
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

dtype = ( None )
load_in_4bit = False
load_in_8bit = False

### List of models
#unsloth/Llama-3.2-3B-Instruct

######---Parameters to change---#######
kaggle_model_path="/kaggle/input/llama-3.2/transformers/1b-instruct/1"
local_model_path="unsloth/Llama-3.2-1B-Instruct"

max_seq_length = 1024
Rank=64
sample_len=int(df_train.shape[0])
max_iter_steps=-1
Epochs=5

## To upload to kagglehub (model name & variation version)
model_slug="llama-3p2-1b-instruct-jigsaw-acrc"
variation_slug="02"
###--------------------------------###


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-03 22:34:29 [__init__.py:235] Automatically detected platform cuda.


## Load model using unsloth

In [3]:
train_parameters=f"_lora_fp16_r{Rank}_s{sample_len}_e_{Epochs}_msl{max_seq_length}"

if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    model_path=kaggle_model_path
else:
    model_path=local_model_path
        
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    load_in_8bit=load_in_8bit
)

print(model.dtype)


==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.1. vLLM: 0.10.0.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.693 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
torch.bfloat16


## Set LoRA Config

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = Rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 123,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.1 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


## Prepare the dataset from train-data

In [5]:
from datasets import Dataset

def formatting_prompts_func(examples):
    """
    Format Reddit moderation dataset for Alpaca training - matches inference format exactly
    """
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request.
### Instruction:
{}
### Input:
{}
### Response:
{}"""
    
    def format_comment(comment_data):
        return comment_data
    
    texts = []
    
    for i in range(len(examples['subreddit'])):
        # Create instruction - exactly as in inference
        instruction = f"""You are a really experienced moderator for the subreddit /r/{examples['subreddit'][i]}. 
Your job is to determine if the following reported comment violates the given rule.
Answer with only "True" or "False"."""
        
        # Create input - exactly as in inference
        input_text = f"""Rule: {examples['rule'][i]}
Example 1:
{format_comment(examples['positive_example_1'][i])}
Rule violation: True
Example 2:
{format_comment(examples['negative_example_1'][i])}
Rule violation: False
Example 3:
{format_comment(examples['positive_example_2'][i])}
Rule violation: True
Example 4:
{format_comment(examples['negative_example_2'][i])}
Rule violation: False
Test sentence:
{format_comment(examples['body'][i])}"""
        
        # Create response - convert 0/1 to probability format
        rule_violation = examples['rule_violation'][i]
        if rule_violation == 1:
            response = "Rule violation: True"
        else:
            response = "Rule violation: False"
        
        # Format the complete prompt
        text = alpaca_prompt.format(instruction, input_text, response) + tokenizer.eos_token
        texts.append(text)
    
    return {"text": texts}

# Apply to your datasets
dataset_train = Dataset.from_pandas(df_train)
dataset_train = dataset_train.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/2029 [00:00<?, ? examples/s]

In [6]:
# Check dataset sample output
dataset_train['text'][0]

'Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request.\n### Instruction:\nYou are a really experienced moderator for the subreddit /r/Futurology. \nYour job is to determine if the following reported comment violates the given rule.\nAnswer with only "True" or "False".\n### Input:\nRule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.\nExample 1:\nIf you could tell your younger self something different about sex, what would that be?\n\ni AM IN A CONTEST TO WIN FUNDING FOR MY SEX POSITIVE FILM: VOTE HERE:\n\nhttp://sheknows.offerpop.com/campaign/813112/entry/v144417\nRule violation: True\nExample 2:\nWatch Golden Globe Awards 2017 Live Online in HD Coverage without ADS (VIP STREAMS)\n=\n\nHD STREAM QUALITY >>> [WATCH LINK1](http://forum.submitexpress.com/viewtopic.php?f=9&t=215858)\n=\n\nHD BROADCASTING QUALITY >>> [WATCH LINK1]

## Setup SFT trainer & train the model

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    #eval_dataset = dataset_test,  # Add test dataset here
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = Epochs, 
        max_steps = max_iter_steps,
        learning_rate = 5e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit", # "adamw_torch" better for fp16
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 123,
        #eval_strategy = "steps", 
        #eval_steps = 100, 
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/2029 [00:00<?, ? examples/s]

In [8]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,029 | Num Epochs = 5 | Total steps = 1,270
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 45,088,768 of 1,280,903,168 (3.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,3.130400
20,2.298000
30,2.181400
40,2.063600
50,1.953200
60,1.817400
70,1.682900
80,1.587700
90,1.448400
100,1.368700


TrainOutput(global_step=1270, training_loss=0.4025115434579023, metrics={'train_runtime': 645.7978, 'train_samples_per_second': 15.709, 'train_steps_per_second': 1.967, 'total_flos': 2.440881292701696e+16, 'train_loss': 0.4025115434579023})

## Save model in merged FP16 format (vllm compatible for inference)

In [9]:
#save merged 16bit
import os
dir_path = "tmp"
os.makedirs(dir_path, exist_ok=True)
model.save_pretrained_merged(dir_path, tokenizer, save_method = "merged_16bit")

Found HuggingFace hub cache directory: /home/vino/.cache/huggingface/hub
Checking cache directory for required files...
Successfully copied all 1 files from cache to tmp.


Unsloth: Merging weights into 16bit: 100%|████████| 1/1 [00:03<00:00,  3.82s/it]


In [10]:
## You may need this login if you want to upload model to kagglehub from local machine.
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    pass
else:
    kagglehub.login()

## Upload model to kaggle

In [1]:
# # Replace with path to directory containing model files.
# LOCAL_MODEL_DIR = dir_path

# MODEL_SLUG = model_slug # Replace with model slug.

# # Learn more about naming model variations at
# # https://www.kaggle.com/docs/models#name-model.
# VARIATION_SLUG = variation_slug # Replace with variation slug.

# kagglehub.model_upload(
#   handle = f"vinothkumarsekar89/{MODEL_SLUG}/transformers/{VARIATION_SLUG}",
#   local_model_dir = LOCAL_MODEL_DIR,
#   version_notes = 'Update 2025-08-02')